This Note Book is same as app.py file
I just used Note Book features to implement its functions :)

In [1]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
from PIL import Image
import io
# from ultralytics.yolo.utils.plotting import Annotator, colors
from ultralytics.utils.plotting import Annotator, colors

WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [2]:
model = YOLO('models/yolov8n.pt')

In [6]:
result = model.predict(source='bus.jpg', conf=0.6)


image 1/1 e:\projects\Python\YOLO-FastAPI\bus.jpg: 640x480 3 persons, 1 bus, 184.0ms
Speed: 23.0ms preprocess, 184.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [23]:
result[0].names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [11]:
result[0].to('cpu').numpy().boxes.xyxy

array([[     22.871,      231.28,         805,      756.84],
       [      48.55,      398.55,      245.35,       902.7],
       [     669.47,      392.19,      809.72,      877.04],
       [     221.52,       405.8,      344.97,      857.54]], dtype=float32)

In [14]:
bboxes = pd.DataFrame(result[0].to('cpu').numpy().boxes.xyxy, columns=['x1', 'y1', 'x2', 'y2'])

In [17]:
bboxes['confs'] = result[0].to('cpu').numpy().boxes.conf

In [20]:
bboxes['classes'] = (result[0].to('cpu').numpy().boxes.cls).astype(int)

In [26]:
bboxes['classes'] = bboxes['classes'].replace(result[0].names)

In [27]:
bboxes

,x1,y1,x2,y2,confs,classes
0,22.871269,231.277313,805.002686,756.840454,0.873449,bus
1,48.550465,398.552216,245.345596,902.702698,0.865691,person
2,669.472900,392.185974,809.720032,877.035461,0.852835,person
3,221.517288,405.798645,344.970612,857.536621,0.825224,person


In [2]:
# Key function 1

model = YOLO('models/yolov8n.pt')
def get_predict_df(file : Image) -> pd.DataFrame:
    # model = YOLO('models/yolov8n.pt')

    result = model.predict(source=file, conf=0.6)
    bboxes = pd.DataFrame(result[0].to('cpu').numpy().boxes.xyxy, columns=['x1', 'y1', 'x2', 'y2'])
    bboxes['conf'] = result[0].to('cpu').numpy().boxes.conf
    bboxes['class'] = (result[0].to('cpu').numpy().boxes.cls).astype(int)
    bboxes['name'] = bboxes['class'].replace(result[0].names)
    
    return bboxes

In [4]:
img = Image.open('bus.jpg').convert("RGB")

In [5]:
preds = get_predict_df(img)


0: 640x480 3 persons, 1 bus, 209.0ms
Speed: 8.0ms preprocess, 209.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [6]:
out_img = draw_boxes(img=img, preds=preds)

In [7]:
out_img.show()

In [5]:
annotator = Annotator(np.array(img))

In [6]:
preds = preds.sort_values(by=['x1'], ascending=True)

In [7]:
for i, data in preds.iterrows():
    # print(f"index= {i}")
    # print(f"data:\n{data}")
    # print('##########\n')
    annotator.box_label(
        box=[
            data['x1'],data['y1'],
            data['x2'],data['y2']
        ],
        label=f"{data['name']}: {data['conf']:.2f}", 
        color=colors(data['class'], bgr=True))

In [8]:
final_img2 = Image.fromarray(annotator.result())

In [9]:
final_img2.show()

In [3]:
# Key function 2

def draw_boxes(img : Image, preds:pd.DataFrame) -> Image:
    annotator = Annotator(np.array(img))
    preds = preds.sort_values(by=['x1'], ascending=True)
    for i, data in preds.iterrows():
        annotator.box_label(
            box=[
                data['x1'],data['y1'],
                data['x2'],data['y2']
            ],
            label=f"{data['name']}: {data['conf']:.2f}", 
            color=colors(data['class'], bgr=True))
    return Image.fromarray(annotator.result())